In [1]:
import json

coco_caption_path = "/home/why/data/caption/dataset_flickr30k.json"


with open(coco_caption_path,'r') as j:
    captions = json.load(j)
    

In [42]:
captions['images'][0]

{'sentids': [0, 1, 2, 3, 4],
 'imgid': 0,
 'sentences': [{'tokens': ['two',
    'young',
    'guys',
    'with',
    'shaggy',
    'hair',
    'look',
    'at',
    'their',
    'hands',
    'while',
    'hanging',
    'out',
    'in',
    'the',
    'yard'],
   'raw': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
   'imgid': 0,
   'sentid': 0},
  {'tokens': ['two',
    'young',
    'white',
    'males',
    'are',
    'outside',
    'near',
    'many',
    'bushes'],
   'raw': 'Two young, White males are outside near many bushes.',
   'imgid': 0,
   'sentid': 1},
  {'tokens': ['two',
    'men',
    'in',
    'green',
    'shirts',
    'are',
    'standing',
    'in',
    'a',
    'yard'],
   'raw': 'Two men in green shirts are standing in a yard.',
   'imgid': 0,
   'sentid': 2},
  {'tokens': ['a',
    'man',
    'in',
    'a',
    'blue',
    'shirt',
    'standing',
    'in',
    'a',
    'garden'],
   'raw': 'A man in a blue shirt standing in

In [50]:
for i in captions['images']:
    if i['imgid'] == 20:
        for j in i['sentences']:
            print(j['raw'])

A person in gray stands alone on a structure outdoors in the dark.
A large structure has broken and is laying in a roadway.
A man in a gray coat is standing on a washed out bridge.
A man stands on wooden supports and surveys damage.
A man in a jacket and jeans standing on a bridge.


# 数据处理

In [2]:
all_document = []


"""
数据结构

all_documents = [{ ‘name’: 图像名字， ‘document’: 图像的5句标注 } ,.....]

"""

for i in captions["images"]:
    d = {}
    d['name'] = i['filename']
    caps = []
    for j in i['sentences']:
        caps.append(j['raw'])
    caps = ' '.join(caps)
    d['document'] = caps
    all_document.append(d)

In [3]:
image_name = []
documents = []
for i in all_document:
    image_name.append(i['name'])
    documents.append(i['document'])

In [4]:
image_name[:10]

['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg',
 '1000523639.jpg',
 '1000919630.jpg',
 '10010052.jpg',
 '1001465944.jpg',
 '1001545525.jpg']

In [5]:
documents[:10]

['Two young guys with shaggy hair look at their hands while hanging out in the yard. Two young, White males are outside near many bushes. Two men in green shirts are standing in a yard. A man in a blue shirt standing in a garden. Two friends enjoy time spent together.',
 'Several men in hard hats are operating a giant pulley system. Workers look down from up above on a piece of equipment. Two men working on a machine wearing hard hats. Four men on top of a tall structure. Three men on a large rig.',
 'A child in a pink dress is climbing up a set of stairs in an entry way. A little girl in a pink dress going into a wooden cabin. A little girl climbing the stairs to her playhouse. A little girl climbing into a wooden playhouse. A girl going into a wooden building.',
 'Someone in a blue shirt and hat is standing on stair and leaning against a window. A man in a blue shirt is standing on a ladder cleaning a window. A man on a ladder cleans the window of a tall building. Man in blue shirt a

# LDA

In [6]:
import json
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import lda

In [7]:
vectorizer = CountVectorizer(analyzer = 'word',stop_words = 'english', lowercase = True, min_df = 5)
X = vectorizer.fit_transform(documents)
word = vectorizer.get_feature_names()       # 词典

In [8]:
len(word)

6844

In [9]:
with open('data/f30_all_word.json','w') as j:
    json.dump(word,j)

In [10]:
analyze = vectorizer.build_analyzer()
weight = X.toarray()

In [11]:
model = lda.LDA(n_topics = 60,n_iter = 500,random_state = 1)
model.fit(np.asarray(weight))

INFO:lda:n_documents: 31014
INFO:lda:vocab_size: 6844
INFO:lda:n_words: 993402
INFO:lda:n_topics: 60
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -12460232
INFO:lda:<10> log likelihood: -7739942
INFO:lda:<20> log likelihood: -6994681
INFO:lda:<30> log likelihood: -6770223
INFO:lda:<40> log likelihood: -6661590
INFO:lda:<50> log likelihood: -6601195
INFO:lda:<60> log likelihood: -6559865
INFO:lda:<70> log likelihood: -6526785
INFO:lda:<80> log likelihood: -6500779
INFO:lda:<90> log likelihood: -6486060
INFO:lda:<100> log likelihood: -6475012
INFO:lda:<110> log likelihood: -6467049
INFO:lda:<120> log likelihood: -6459326
INFO:lda:<130> log likelihood: -6452845
INFO:lda:<140> log likelihood: -6447079
INFO:lda:<150> log likelihood: -6445293
INFO:lda:<160> log likelihood: -6439721
INFO:lda:<170> log likelihood: -6434849
INFO:lda:<180> log likelihood: -6430013
INFO:lda:<190> log likelihood: -6424200
INFO:lda:<200> log likelihood: -6423370
INFO:lda:<210> log likelihood: -6417042
INFO:lda

In [12]:
'''每个主题的单词比重'''

topic_word = model.topic_word_
print(topic_word)

[[7.96340655e-07 7.96340655e-07 7.96340655e-07 ... 7.96340655e-07
  7.96340655e-07 7.96340655e-07]
 [6.66025506e-07 6.66025506e-07 6.66025506e-07 ... 6.66025506e-07
  6.66025506e-07 6.66025506e-07]
 [7.25455659e-07 7.25455659e-07 7.25455659e-07 ... 7.25455659e-07
  7.25455659e-07 7.25455659e-07]
 ...
 [6.43151339e-07 6.43151339e-07 6.43151339e-07 ... 6.43151339e-07
  6.43151339e-07 6.43151339e-07]
 [3.23494360e-07 3.23494360e-07 3.23494360e-07 ... 3.23494360e-07
  3.23494360e-07 3.23494360e-07]
 [4.73584840e-04 7.87994742e-07 7.87994742e-07 ... 7.87994742e-07
  7.95874690e-05 7.87994742e-07]]


In [13]:
len(topic_word)

60

In [14]:
len(topic_word[0])

6844

In [15]:
topic_word = topic_word.tolist()

In [16]:
with open('f30_per_topic_worddiscrim.json','w') as j:
    json.dump(topic_word,j)

In [17]:
"""每个主题中最重要的20个单词"""
n = 20
N_top_word = []
for i, word_weight in enumerate(topic_word):
    distIndexArr = np.argsort(word_weight)
    topN_index = distIndexArr[:-(n+1):-1]
    topN_words = np.array(word)[topN_index]
    s = "*Topic "+str(i)+": "+' '.join(topN_words)
    N_top_word.append(s)
    #print('*Topic {}\n- {}'.format(i, ' '.join(topN_words)))

In [18]:
N_top_word

['*Topic 0: snow snowy person covered hill slide winter snowboarder jacket skier mountain skiing ski sled coat wearing red cold skis sliding',
 '*Topic 1: camera picture taking smiling posing photo pose takes standing man pictures young poses holding smiles smile photographer photograph looking taken',
 '*Topic 2: man carrying bag cart pushing bags pulling large box shopping head carries basket plastic pushes street person trash garbage shoulder',
 '*Topic 3: woman wearing black white dress lady shirt blue pink purple girl skirt tank young blond female purse sunglasses women dressed',
 '*Topic 4: water beach pool sand swimming fountain ocean bathing swim wet goggles suit trunks diving near shorts sandy blue splashing playing',
 '*Topic 5: dog mouth black white ball brown stick toy frisbee catch playing carrying cat small jumps jumping orange holding tennis object',
 '*Topic 6: man drinking drink having bar holding beer laughing cup glass bottle drinks coffee smiling young sitting party

In [19]:
with open('f30_Topic_i_20_word.json','w') as j:
    json.dump(N_top_word,j)

In [20]:
"""获取每个主题的单词表示"""

word_20 = []

for i in N_top_word:
    i = i.split(': ')[1]
    word_20.append(i)

In [21]:
word_20

['snow snowy person covered hill slide winter snowboarder jacket skier mountain skiing ski sled coat wearing red cold skis sliding',
 'camera picture taking smiling posing photo pose takes standing man pictures young poses holding smiles smile photographer photograph looking taken',
 'man carrying bag cart pushing bags pulling large box shopping head carries basket plastic pushes street person trash garbage shoulder',
 'woman wearing black white dress lady shirt blue pink purple girl skirt tank young blond female purse sunglasses women dressed',
 'water beach pool sand swimming fountain ocean bathing swim wet goggles suit trunks diving near shorts sandy blue splashing playing',
 'dog mouth black white ball brown stick toy frisbee catch playing carrying cat small jumps jumping orange holding tennis object',
 'man drinking drink having bar holding beer laughing cup glass bottle drinks coffee smiling young sitting party glasses enjoying pouring',
 'woman hair talking blond cellphone long 

In [22]:
with open('f30_word_20.json','w') as j:
    json.dump(word_20,j)

In [23]:
'''判断文本的主题概率，为每个文本打上主题标签'''
doc_topic = model.doc_topic_
print("type(doc_topic):{}".format(type(doc_topic)))
print("shape:{}".format(doc_topic.shape))
label = []
for i in range(10):
    print(doc_topic[i])
    topic_most_pr = doc_topic[i].argmax()
    label.append(topic_most_pr)
    print("doc:{}topic:{}".format(i,topic_most_pr))
    
print(label)

type(doc_topic):<class 'numpy.ndarray'>
shape:(31014, 60)
[0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.06363636 0.0030303  0.03333333 0.0030303  0.03333333 0.0030303
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.0030303  0.06363636 0.0030303  0.0030303  0.30606061 0.21515152
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.0030303
 0.0030303  0.0030303  0.0030303  0.0030303  0.0030303  0.06363636
 0.0030303  0.06363636 0.0030303  0.0030303  0.0030303  0.0030303 ]
doc:0topic:34
[0.00357143 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143
 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143
 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143
 0.00357143 0.00357143 0.00357143 0.00357143 0.00357143 0.00357

In [24]:
doc_topic = doc_topic.tolist()


In [25]:
len(doc_topic)

31014

In [26]:
len(doc_topic[0])

60

In [27]:
with open('f30_topic_distribution.json','w') as j:
    json.dump(doc_topic,j)

In [27]:
# import json

# with open('topic_distribution.json','r') as j:
#     doc_topic = json.load(j)


In [28]:
max_top = []
for i in doc_topic:
    m = i.index(max(i))
    max_top.append(m)


In [31]:
#max_top

In [35]:
dname = []

for i in image_name:
    i = i.split('.')[0]
    #i = int(i.lstrip("0").split('.')[0])
    dname.append(i)

In [31]:
# dname

In [37]:
name_top = {}
name_top_word = {}

for n,i in enumerate(dname):
    
    name_top[i] = max_top[n]
    name_top_word[i] = word_20[max_top[n]]

In [33]:
"""数据结构：{image_name:top_1_topic}"""

# name_top

'数据结构：{image_name:top_1_topic}'

In [38]:
with open('f30_top_1_topic_class.json','w') as j:
    json.dump(name_top,j)
with open('f30_top_1_topic_word.json','w') as j:
    json.dump(name_top_word,j)

In [41]:
name_top_word

{'1000092795': 'men group standing people shirts stand guys talking hats wearing white suits dressed black jackets uniforms having males conversation talk',
 '10002456': 'working construction workers orange work hard worker men wearing yellow vests roof safety vest building site hats equipment fixing machine',
 '1000268201': 'girl young little pink child dress wearing small blond purple shirt hula hair polka plays daughter smiling blue haired toddler',
 '1000344755': 'building outside standing window wall brick stands door near house large stone leaning old white doorway windows blue glass inside',
 '1000366164': 'food eating kitchen restaurant cooking preparing apron table grill white meal bowl meat eat chef counter eats plate making dinner',
 '1000523639': 'man guitar playing microphone singing stage plays black shirt sings electric singer white holding background performing male acoustic piano musician',
 '1000919630': 'face statue head hair making cake bubbles blowing young mirror 

In [39]:
len(name_top_word)

31014

In [1]:
import os
import numpy as np
import h5py
import json
import torch
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import pickle


with open('/home/why/data/caption/dataset_flickr30k.json', 'r') as j:
    data = json.load(j)
    
with open("f30_top_1_topic_class.json", 'r') as j:
    image_topclass= json.load(j)


/home/why/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_topic = []
val_topic = []
test_topic = []

test_name = []


len_data = len(data['images'])
for num, img in enumerate(data['images']):
    
    image_id = img['filename'].split('.')[0]

    if img['split'] in {'train', 'restval'}:
        
        top_class = image_topclass[str(image_id)]
        
        
        train_topic.append(top_class)

    elif img['split'] in {'val'}:
        
        top_class = image_topclass[str(image_id)]
        
        val_topic.append(top_class)
  
    elif img['split'] in {'test'}:
        top_class = image_topclass[str(image_id)]
        
        test_name.append(image_id)
        test_topic.append(top_class)
 

In [3]:
with open('F30K_TEST_NAME.json', 'w') as j:
    json.dump(test_name, j)

In [6]:
with open('F30K_TRAIN_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(train_topic, j)

with open('F30K_VAL_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(val_topic, j)
    
with open('F30K_TEST_1_TOPIC_CLASS.json', 'w') as j:
    json.dump(test_topic, j)


# with open('TRAIN_1_TOPIC_WORD.json', 'w') as j:
#     json.dump(train_topic_word, j)

# with open('VAL_1_TOPIC_WORD.json', 'w') as j:
#     json.dump(val_topic_word, j)
    
# with open('TEST_1_TOPIC_WORD.json', 'w') as j:
#     json.dump(test_topic_word, j)


In [41]:
topic_class = []

topic_word = []



len_data = len(data['images'])
for num, img in enumerate(data['images']):
    
    image_id = img['filename'].split('_')[2]
    image_id = int(image_id.lstrip("0").split('.')[0])

    top_class = image_topclass[str(image_id)]
    word = image_topword[str(image_id)]
    word = word.split(' ')
    enc_w = [word_map.get(word, word_map['<unk>']) for word in word]

    topic_class.append(top_class)
    topic_word.append(enc_w)


In [43]:
with open('TOPIC_1_CLASS.json', 'w') as j:
    json.dump(topic_class, j)


with open('TOPIC_1_WORD.json', 'w') as j:
    json.dump(topic_word, j)
